In [ ]:
import torch
import torchaudio
import librosa
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import HubertForSequenceClassification, Wav2Vec2Processor, Trainer, TrainingArguments

# Define dataset class
class MachineSoundDataset(Dataset):
    def __init__(self, data_dir, processor, labels):
        self.data_dir = data_dir
        self.processor = processor
        self.labels = labels
        self.files = []
        for label in labels:
            class_path = os.path.join(data_dir, label)
            for file in os.listdir(class_path):
                if file.endswith(".wav"):
                    self.files.append((os.path.join(class_path, file), label))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path, label = self.files[idx]
        
        # Load audio
        waveform, sr = librosa.load(file_path, sr=16000)  # HuBERT expects 16kHz
        input_values = self.processor(waveform, sampling_rate=16000, return_tensors="pt").input_values[0]
        
        return {"input_values": input_values, "labels": torch.tensor(self.labels[label])}

# Define paths
DATA_DIR = "data/trimmed_valve"
LABELS = {"normal": 0, "abnormal": 1}  # Binary classification labels

# Load processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-base-ls960")
model = HubertForSequenceClassification.from_pretrained("facebook/hubert-base-ls960", num_labels=2)

# Load dataset
dataset = MachineSoundDataset(DATA_DIR, processor, LABELS)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./hubert_model",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    evaluation_strategy="steps",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("hubert_machine_sound")
processor.save_pretrained("hubert_machine_sound")


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f9171216220>>
Traceback (most recent call last):
  File "/Users/kingnutmegs/Documents/GitHub/machine-noise-generator/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
/Users/kingnutmegs/Documents/GitHub/machine-noise-generator/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 